<a href="https://colab.research.google.com/github/JozefSL/pyNotes/blob/main/EIA/PacificStorageWklyChanges.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import requests

In [2]:
from google.colab import userdata
api_key = userdata.get('api_key')

In [3]:
url = 'https://api.eia.gov/v2/natural-gas/stor/wkly/data/?api_key='+ api_key + '&frequency=weekly&data[0]=value&start=2019-01-01&sort[0][column]=period&sort[0][direction]=asc&facets[series][]='

In [4]:
response = requests.get(url + 'NW2_EPG0_SWO_R31_BCF')

In [5]:
df = pd.DataFrame(response.json().get('response').get('data'))[['period','value']]
df.columns = ['Period', 'value']
df['Period'] = df['Period'].apply(pd.to_datetime)
df.value = (df.value.astype('Int64')/1).astype('Int64')
df['year'] = df['Period'].dt.year
df['week'] = df['Period'].dt.isocalendar().week
df['change'] = df['value'] - df['value'].shift(1)

df.tail(3)

,Period,value,year,week,change
297,2024-09-13,802,2024,37,22
298,2024-09-20,818,2024,38,16
299,2024-09-27,846,2024,39,28


In [6]:
ty = df.year.iloc[-1]
ly = ty-1

In [7]:
dfA = df.pivot(index='week', columns='year', values='change').astype('Int64') #.reset_index(drop='true')
dfA = dfA.drop(dfA.index[-1]).reset_index()  #eliminate the add years with 53 weeks

In [8]:
dfA.tail(3)

year,week,2019,2020,2021,2022,2023,2024
47,48,-3,0,-24,-6,-39,<NA>
48,49,-24,-19,-25,-12,-19,<NA>
49,50,-29,-34,-9,-33,-23,<NA>
50,51,-42,-37,-32,-42,-29,<NA>
51,52,-25,-34,-10,-56,-6,<NA>


In [9]:
dfAA = pd.DataFrame(dfA.iloc[:, -6:-1].mean(axis=1))
dfAA.columns = ['5yAvg']
dfAA['5yMin'] = dfA.iloc[:, -6:-1].min(axis=1) # Actual minimum, also used to calculate stack area bottom
dfAA['5yMaxA'] = dfA.iloc[:, -6:-1].max(axis=1) # Actual maximum, also used to calculate stack area top
dfAA['5yMax'] = dfAA['5yMaxA']-dfAA['5yMin']
#dfAA['10yAvg'] = dfA.iloc[:, -11:-1].mean(axis=1)
#dfAA['10yMin'] = dfA.iloc[:, -11:-1].min(axis=1)
#dfAA['10yMax'] = dfA.iloc[:, -11:-1].max(axis=1)
dfAA['week'] = dfA['week']

In [10]:
dfAA.head(3)

,5yAvg,5yMin,5yMaxA,5yMax,week
0,-20.5,-39,9,48,1
1,-43.4,-61,-31,30,2
2,-42.4,-60,-20,40,3


In [11]:
dfA['weekDate'] = pd.to_datetime(str(pd.Timestamp.now().year)+dfA['week'].astype(str)+'5',format='%Y%W%w')
dfAA['weekDate'] = pd.to_datetime(str(pd.Timestamp.now().year)+dfAA['week'].astype(str)+'5',format='%Y%W%w')

In [12]:
categories = dfAA['weekDate']
lower_bounds = dfAA['5yMin']
upper_bounds = dfAA['5yMaxA']

In [14]:
fig = go.Figure()

for i in range(len(categories)):
    fig.add_trace(go.Bar(
        x=[categories[i]],
        y=[upper_bounds[i] - lower_bounds[i]],
        base=[lower_bounds[i]],
        marker=dict(color='lightgrey'),
        showlegend=False
        #color_discrete_sequence=['blue']
        #name=str(categories[i])
    ))

fig.add_scatter(x=dfAA.weekDate, y=dfAA['5yAvg'], mode='markers', name='5yAvg', line=dict(color='grey', width=2))
fig.add_scatter(x=dfA.weekDate, y=dfA[ly], mode='markers', name=str(ly),line=dict(color='blue', width=2))
fig.add_scatter(x=dfA.weekDate, y=dfA[ty], mode='lines+markers', name=str(ty),line=dict(color='red', width=3), marker=dict(color='red'))

fig.add_scatter(x = [dfA['weekDate'][dfA[ty].isna().idxmax()-1]], y = [dfA[ty][dfA[ty].isna().idxmax()-1]],
                     mode = 'markers + text',
                     marker = {'color':'red', 'size':14},
                     showlegend = False,
                     text = [dfA[ty][dfA[ty].isna().idxmax()-1]],
                     textfont=dict(color='red', size=16),
                     textposition='top right') # top, bottom, middle

fig.update_layout(autosize=False,width=1200, height=700,
    title={
        'text': '<b>East Region Working Inventory Weekly Changes</b>',
        'font': {'size': 28, 'family': 'Arial', 'color': 'black'}
        },
    #xaxis_title='Week',
    yaxis_title='East Region Working Inventory Weekly Change (Bcf)')

# Add source link as an annotation
fig.add_annotation(
    text="Source: EIA     https://ir.eia.gov/ngs/ngs.html",
    xref="paper", yref="paper",
    x=0.95, y=0.05,
    showarrow=False,
    font=dict(size=13)
)

fig.show()